In [13]:
#import modules
import json
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv
from envs import env
import time
import re
import os
import datetime
from datetime import timedelta
import statistics
import numpy as np
from scipy.stats import kurtosis, skew

In [14]:
#import classes from ./ folder
import postgresdbAccess

In [15]:
#read fibonacci retracements  from json
with open('fibLvl.json') as file:
    fibLvl = json.load(file)

In [16]:
def writeTrade(openPositions, row, postgres):
    openPositions['resultPercent'] = (((row[4] - 
        openPositions['askPrice']) / openPositions['askPrice']) * 
        100 - brokerFees * 2.55)
    openPositions['stopId'] = row[0]        
    closeSql = ("UPDATE " + dbTable + " SET" +
        " startId = '" + str(int(openPositions['startId'])) +
        "', fibLevel = '" + str(openPositions['fibLevel']) +
        "', midId = '" + str(int(openPositions['id'])) +
        "', corValue = '" + str(openPositions['corValue']) +
        "', takeProfit = '" + str(openPositions['takeProfit']) +
        "', stopLoss = '" + str(openPositions['stopLoss']) +  
        "', resultpercent = '" + str(openPositions['resultPercent']) +
        "', stopid = '" + str(openPositions['stopId']) +
        "', skew = '" + str(openPositions['skew']) +
        "', kurtosis = '" + str(openPositions['kurtosis']) +
        "' WHERE id = '" + str(int(openPositions['id'])) +
        "';")
    postgres.sqlUpdate(closeSql)

In [17]:
def backtest():
    #initiate empty open positions
    openPositions = {}
    #initiate postgresdb class
    postgres = postgresdbAccess.postgresAccess()
    #get unique simbol list
    sql = ("SELECT distinct symbol FROM " + dbTable + ";")
    uniqueSymbol = pd.DataFrame(postgres.sqlQuery(sql))
    uniqueSymbol = uniqueSymbol[0]
    #initiate empty percent bar
    percentBar = 0
    #loop over every symbol in symbol list
    for symbol in uniqueSymbol:
        #print progress
        print(percentBar, " / ", len(uniqueSymbol))
        percentBar = percentBar + 1
        #skip if symbol is not to be considered
        #null askprice is sorted out with crawler already
        if not (symbol.endswith(baseCurrency) and
               len(symbol) < 11):
            continue
        #check if open position was left over from last symbol and simulate close
        if openPositions:
            writeTrade(openPositions, row, postgres)
            openPositions = {}
        #query for every line for symbol
        sql = ("SELECT id, time, symbol, askprice, " +
               "bidprice, highprice, quotevolume, " +
               "pricechangepercent, weightedavgprice, openprice" +
            " FROM " + dbTable + " WHERE" +
            " symbol like '" + symbol + "' order by id;")
        print(sql)
        bigData = pd.DataFrame(postgres.sqlQuery(sql))
        bigData[0] = pd.to_numeric(bigData[0], errors='coerce', downcast='float')
        bigData[1] = pd.to_datetime(bigData[1])
        bigData[3] = pd.to_numeric(bigData[3], errors='coerce', downcast='float')
        bigData[4] = pd.to_numeric(bigData[4], errors='coerce', downcast='float')
        bigData[6] = pd.to_numeric(bigData[6], errors='coerce', downcast='float')
        bigData[7] = pd.to_numeric(bigData[7], errors='coerce', downcast='float')
        bigData[8] = pd.to_numeric(bigData[8], errors='coerce', downcast='float')
        bigData[9] = pd.to_numeric(bigData[9], errors='coerce', downcast='float')
        #get start of timedelta
        bigData[5] = bigData[1] - timedelta(hours=33)
        bigData[10] = bigData[1] - timedelta(hours=22)
        bigData[11] = bigData[1] - timedelta(hours=11)
        #loop over every row
        for index, row in bigData.iterrows():
            before_start_date = bigData[1] <= row[5]
            if len(bigData.loc[before_start_date]) > 0:
                #get data for consideration
                after_start_date = bigData[1] >= row[5]
                before_end_date = bigData[1] <= row[1]
                between_two_dates = after_start_date & before_end_date
                fibDates = bigData.loc[between_two_dates]
                diff = fibDates[3].max() - fibDates[3].min()
                # calculate fibRetracements
                fibRetracement = pd.DataFrame(fibLvl)
                maxAsk = fibDates[3].max()
                for lvl in fibRetracement:
                    fibRetracement[1] =  maxAsk - diff * fibRetracement[0]
                    fibRetracement[2] = fibRetracement[1] * 0.9995
                    fibRetracement[3] = fibRetracement[1] * 1.0005
                #calculate corvalue
                corValue = fibDates[0].corr(fibDates[3])
                before_start_date1 = bigData[1] <= row[5]
                bigData[10] = bigData[1] - timedelta(hours=15)
                after_start_date1 = bigData[1] >= row[10]
                before_end_date1 = bigData[1] <= row[1]
                between_two_dates1 = after_start_date1 & before_end_date1
                fibDates1 = bigData.loc[between_two_dates]
                corValue1 = fibDates1[0].corr(fibDates1[3])
                
                before_start_date2 = bigData[1] <= row[5]
                bigData[11] = bigData[1] - timedelta(hours=5)
                after_start_date2 = bigData[1] >= row[11]
                before_end_date2 = bigData[1] <= row[1]
                between_two_dates2 = after_start_date2 & before_end_date2
                fibDates2 = bigData.loc[between_two_dates2]
                corValue2 = fibDates2[0].corr(fibDates2[3])
                
                #get statistical parameters
                statisticsTools = {}
                statisticsTools["stDev"] = statistics.stdev(fibDates[3])
                statisticsTools["skew"] = skew(fibDates[3])
                statisticsTools["kurtosis"] = kurtosis(fibDates[3])
                
                #if an open position exists, check if it can be closed
                if openPositions:
                    if (row[3] < openPositions['stopLoss'] or
                        row[3] > openPositions['takeProfit']):
                        writeTrade(openPositions, row, postgres)
                        #clear open position
                        openPositions = {}
                else:
                    #loop over considered fibonacciretracements
                    for i in [7]:
                        #check if buy requirements are met
                        if (corValue >= 0 and
                            corValue1 >= 0 and
                            corValue2 >= 0 and
                            row[3] < fibRetracement[3][i] and
                            row[3] > fibRetracement[2][i]):
                            openPositions['startId'] = fibDates[0].min()
                            openPositions['id'] = row[0]
                            openPositions['bidPrice'] = row[4]
                            openPositions['fibLevel'] = i
                            openPositions['symbol'] = symbol
                            openPositions['askPrice'] = row[3]
                            openPositions['corValue'] = corValue
                            openPositions['stDev'] = statisticsTools["stDev"]
                            openPositions['skew'] = statisticsTools["skew"]
                            openPositions['kurtosis'] = statisticsTools["kurtosis"]
                            openPositions['takeProfit'] = fibRetracement[3][i+4]
                            openPositions['stopLoss'] = fibRetracement[2][i-1]
    #close database connection
    postgres.databaseClose()

In [ ]:
testTables = [{"table":"backtesting","currency":"BNB"},
             {"table":"backtesting","currency":"BTC"},
             {"table":"backtesting","currency":"ETH"},
             {"table":"backtesting","currency":"ADA"}]
for iteration in testTables:
    load_dotenv('../.env')
    try:
        liveVolume=env("liveVolume")
        apiSecret=env('apiSecret')
        apiKey=env('apiKey')
        dbTable=iteration["table"]
        baseCurrency=iteration["currency"]
        brokerFees=float(env('brokerFees'))
        liveTrading=env("liveTrading", 'False').lower() in ('true', '1', 't')
    except KeyError:
        print("No env variables set.")
        sys.exit(1)
    client = Client(apiKey, apiSecret, {'timeout':600})
    backtest()

0  /  752
1  /  752
2  /  752
3  /  752
4  /  752
5  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ADXBNB' order by id;
6  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ENJBNB' order by id;
7  /  752
8  /  752
9  /  752
10  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CNDBNB' order by id;
11  /  752
12  /  752
13  /  752
14  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'WANBNB' order by id;
15  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHER

116  /  752
117  /  752
118  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CVCBNB' order by id;
119  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'TORNBNB' order by id;
120  /  752
121  /  752
122  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'XVSBNB' order by id;
123  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'FLMBNB' order by id;
124  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MLNBNB' order by id;
125  /  752
SELECT id

225  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MINABNB' order by id;
226  /  752
227  /  752
228  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'KP3RBNB' order by id;
229  /  752
230  /  752
231  /  752
232  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'NUBNB' order by id;
233  /  752
234  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ANKRBNB' order by id;
235  /  752
236  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MBLBN

350  /  752
351  /  752
352  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'CREAMBNB' order by id;
353  /  752
354  /  752
355  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'WINBNB' order by id;
356  /  752
357  /  752
358  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'WINGBNB' order by id;
359  /  752
360  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'ONTBNB' order by id;
361  /  752
362  /  752
363  /  752
364  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM

491  /  752
492  /  752
493  /  752
494  /  752
495  /  752
496  /  752
497  /  752
498  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'RCNBNB' order by id;
499  /  752
500  /  752
501  /  752
502  /  752
503  /  752
504  /  752
505  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'FETBNB' order by id;
506  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'TCTBNB' order by id;
507  /  752
508  /  752
509  /  752
510  /  752
511  /  752
512  /  752
513  /  752
514  /  752
515  /  752
516  /  752
517  /  752
518  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE sy

620  /  752
621  /  752
622  /  752
623  /  752
624  /  752
625  /  752
626  /  752
627  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'AIONBNB' order by id;
628  /  752
629  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'BURGERBNB' order by id;
630  /  752
631  /  752
632  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'BRDBNB' order by id;
633  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weightedavgprice, openprice FROM backtesting WHERE symbol like 'MASKBNB' order by id;
634  /  752
635  /  752
636  /  752
SELECT id, time, symbol, askprice, bidprice, highprice, quotevolume, pricechangepercent, weight